In [46]:
import urllib.request as request
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf",
                    "data-files/economy-dictionary.pdf")

('data-files/economy-dictionary.pdf',
 <http.client.HTTPMessage at 0x2651098fac0>)

In [4]:
%%time
# pdf 파일의 텍스트 정보를 읽어서 페이지 단위로 반환
loader = PyPDFLoader('data-files/economy-dictionary.pdf')
pages = loader.load_and_split()

CPU times: total: 15.5 s
Wall time: 15.5 s


In [5]:
print( len(pages) )
# print( pages[0].page_content )
# print( pages[12].page_content ) # 0 ~ 12번 문서는 목차
# print( pages[-1].page_content ) # 마지막 문서는 기타 정보

pages2 = pages[13:-1] # 불필요한 문서(페이지) 제거
print(len(pages2))

366
352


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [7]:
documents = text_splitter.split_documents(pages2)

In [8]:
len(documents)

646

In [10]:
# 분할된 문서들을 vectordb에 저장 (여기서는 Chroma)

chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장
# chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(), 
#                                   persist_directory="vectordb/chroma.economy.db") # 파일로 저장

C:\Users\human\AppData\Local\Temp\ipykernel_8064\3814112023.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장


In [11]:
chroma_db._collection.count()

646

In [15]:
# 데이터 저장 구조 (구성) 확인
# for k, v in chroma_db._collection.get().items(): # items() : key, value from dict
#     print(k)

for k in chroma_db._collection.get(): # key from dict
    print(k)

ids
embeddings
documents
uris
data
metadatas
included


In [ ]:
chroma_db._collection.get()["ids"] # 데이터 반환
chroma_db._collection.get()["embeddings"] # 어떤 데이터는 None으로 반환
chroma_db._collection.get(include=["embeddings"])['embeddings']

In [24]:
embeddings = chroma_db._collection.get(include=["embeddings"])['embeddings']

In [26]:
embeddings.shape

(646, 1536)

In [27]:
# 유사도가 높은 2개의 문서를 반환하는 반환기로 설정
retriever = chroma_db.as_retriever(search_kwargs={"k":2}) 

In [42]:
docs = retriever.get_relevant_documents("비트코인을 알려주세요")

In [43]:
print(len(docs))
print(docs[0])

2
page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다른 
속성을 지닌다. 한편 2017년 8월 1일 비트코인(classic bitcoin)에서 ‘비트코인캐시’(BCH)

In [45]:
# 템플릿 만들기
template = """
당신은 한국은행에서 만든 금융 용어를 설명해주는 도우미입니다.
주어진 검색 결과를 기반으로 답변해 주세요.
검색 결과에 없는 내용은 답변할 수 없다고 해주세요.

[검색결과]
{context}

질문: {question}
답변:"""

prompt = PromptTemplate.from_template(template=template)

In [47]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)